In [1]:
#interface graphique 
from tkinter import Tk, Canvas, Button
import pprint
import random
import time
#import de notebook
import nbimporter
from ia import Ia_dumb
Ia_dumb()

Importing Jupyter notebook from ia.ipynb
running


<h2><b>GAME</b></h2>

In [2]:
#function pour dessiner 
def pack_util(p) : 
    return p.pack()

In [3]:
#classe principale 
class Snake_game :
    def __init__(self, tinker, size = 500, play_btn = True):
        #initialisation des variables
        self.game_size = size
        self.pixel_size = 20
        self.tinker = tinker
        self.commands = {
            "z" : "top",
            "q" : "left",
            "d" : "right",
            "s" : "bottom"
        }
        self.reverse_commands = {
            "top" : "bottom",
            "right" : "left",
            "left" : "right",
            "bottom" : "top",
        }
        self.direction = ""
        self.commands_history = [self.direction]
        
        #initialisation fenêtre tkinter
        self.tinker.title("Snake")
        self.tinker.resizable(False, False)
        self.tinker.tk.call("tk", "scaling", 4.0)
        
        #initialisation de la matrice / creation d'une ligne vide
        self.grid_pos = [[] for i in range(0, int(self.game_size / self.pixel_size))]
        self.create_grid()
        
        #booleen pour afficher le bouton play 
        if play_btn : 
            #bouton jouer
            self.play_button = Button(self.tinker, text = "PLAY", command = self.play, width = 8, height = 2)
            pack_util(self.play_button)
            
    def create_canvas(self) :
        #canvas
        canvas = Canvas(self.tinker, width = self.game_size, height = self.game_size, background = "black", highlightthickness = 0)
        return canvas
          
    def create_grid(self) :
        
        #initialisation d'un dictionnaire
        pos_dict = {}             
        #remplir chaque ligne avec une liste de tuple de coordonnées possibles  
        for x in range(0, self.game_size + 1, self.pixel_size) :
            pos_dict[x] = []
            for y in range(0, self.game_size + 1, self.pixel_size) :
                pos_dict[x].append((x, y))

        #création de matrice      
        for x in list(pos_dict.keys())[:-1]:
            for i in range(len(pos_dict[x])-1):
                self.grid_pos[i].append([pos_dict[x][i],pos_dict[x+self.pixel_size][i+1]])
        
    #fonction d'écoute de touche    
    def set_key_event(self, event, canvas) :
        #vérifier la touche 
        if event.char in self.commands :
            #définir la direction
            self.direction = self.commands[event.char]
            
            #on vérifie si l'utilisateur n'appuie pas deux fois sur la même touche
            if (self.commands_history[-1] != self.direction) and (self.commands_history[-1] != self.reverse_commands[self.direction]) : 
                self.commands_history.append(self.direction)
                self.snake.move(self.direction, canvas, self.commands_history)
        
    def play(self) :
        print("Playing")
        #retirer le bouton play 
        self.play_button.destroy()
        
        #initialisation canvas
        self.canvas = self.create_canvas()
        
        #on vérifie si il y a plus de 1 élément fils sur la fenêtre
        if len(self.tinker.winfo_children()) == 1 :
            #initialisation de la pomme
            self.apple = Apple(self.tinker, self.game_size, False)
            self.apple.create_apple(self.canvas)
            
            #initialisation du snake
            self.snake = Snake(self.tinker, self.game_size, False)
            self.snake.create_snake(self.canvas)
            
            #écouter touche du clavier
            self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas))
            
            pack_util(self.canvas)

In [4]:
#TODO set Apple 
class Apple (Snake_game):
    def __init__(self, tinker, game_size, play_btn):
        super().__init__(tinker, game_size, False)
        self.init_pos = int((self.game_size/self.pixel_size)/2)

    def create_apple(self, canvas):
        print("apple", canvas)
        #création de la pomme
        self.apple = canvas.create_rectangle(
                self.grid_pos[self.init_pos][self.init_pos][0][1],
                self.grid_pos[self.init_pos][self.init_pos][0][0],
                self.grid_pos[self.init_pos][self.init_pos][1][1],
                self.grid_pos[self.init_pos][self.init_pos][1][0],
                outline="#000000",
                fill="#ff8f26"
            )

In [5]:
class Snake (Snake_game) :
    def __init__(self, tinker, game_size, play_btn):
        super().__init__(tinker, game_size, False)
        #initialisation position snake
        self.snake_head = (7, 5)
        self.snake_positions = [self.snake_head, (6,5), (5,5)]
        self.direction = "none"
        
    def create_snake(self, canvas):
        print("snake", canvas)
        for i, position in enumerate(self.snake_positions) :
            self.snake_part = canvas.create_rectangle(
                self.grid_pos[position[0]][position[1]][0][1],
                self.grid_pos[position[0]][position[1]][0][0],
                self.grid_pos[position[0]][position[1]][1][1],
                self.grid_pos[position[0]][position[1]][1][0],
                outline="#000000",
                fill="#1EC81C",
                tags=('snake_head') if i == 0 else ('snake')
            )
            
    def move(self, direction, canvas, commands_history) :
        
        #id tête du snake
        self.snake_head_ids = canvas.find_withtag("snake_head")
        
        #tout les ids du corps du snake
        self.snake_parts_ids = canvas.find_withtag("snake")
        
        #id de tout le snake
        self.snake_ids =  self.snake_parts_ids + self.snake_head_ids
        
        #controle de la boucle
        movement = True
        
        #changement de direction
        if len(commands_history) > 2 : 
            self.direction_is_diff = True if (direction == "right") or (direction == "left") else False
        else :
            self.direction_is_diff = False
        
        
        #boucle de mouvement
        while movement :
            for i, snake_id in enumerate(self.snake_ids):
                #coordonné d'une partie du snake par id
                coords = canvas.coords(snake_id)
                print("coords", coords)
                
                draw = False

                x = self.snake_positions[i][0]
                y = self.snake_positions[i][1]

                head_x = self.snake_head[0]
                head_y = self.snake_head[1]
                
                #mouvement à droite
                if direction == "right" :
                    y = head_y
                    #on vérifie que ça ne touche pas les bords 
                    if x < len(self.grid_pos[x]) - 1:
                        if self.direction_is_diff:
                            x += i+1
                            if i == len(self.snake_ids) - 1 :
                                self.direction_is_diff = False
                                print("changed is diff", self.direction_is_diff)
                        else :
                            x += 1
                        self.snake_head = (x,y) if self.snake_positions[i] == self.snake_head else self.snake_head     
                        self.snake_positions[i] = (x,y)
                        draw = True
                    else :
                        movement = False
                        break
                            
                
                if direction == "left" :
                    y = head_y
                    if x < len(self.grid_pos[x]) - 1:
                        if x > 0 :
                            print("x>0")
                            print("self.direction_is_diff", self.direction_is_diff)
                            if self.direction_is_diff :
                                x -= i+1
                                print("Index left",i)
                                if i == len( self.snake_ids) - 1 :
                                    self.direction_is_diff = False
                                    print("changed is diff", self.direction_is_diff)
                            else :
                                x -= 1
                            self.snake_head = (x,y) if self.snake_positions[i] == self.snake_head else self.snake_head 
                            self.snake_positions[i] = (x,y)
                            draw = True
                        else :
                            movement = False
                            break
                    else :
                        movement = False
                        break
                        
                if direction == "bottom" :
                    x = head_x
                    if y < len(self.grid_pos) - 1:
                        if self.direction_is_diff == False:
                            y += i+1
                            print("Index bottom",i)
                            if i == len (self.snake_ids) - 1 :
                                self.direction_is_diff = True
                                print("changed is diff", self.direction_is_diff)
                        else :
                            y += 1
                        self.snake_head = (x,y) if self.snake_positions[i] == self.snake_head else self.snake_head 
                        self.snake_positions[i] = (x,y)
                        draw = True
                    else :
                        movement = False
                        break
                
                if direction == "top" :
                    x = head_x
                    if y > 0:
                        if self.direction_is_diff == False:
                            y -= i+1
                            print("Index bottom",i)
                            if i == len (self.snake_ids) - 1 :
                                self.direction_is_diff = True
                                print("changed is diff", self.direction_is_diff)
                        else :
                            y -= 1
                        self.snake_head = (x,y) if self.snake_positions[i] == self.snake_head else self.snake_head 
                        self.snake_positions[i] = (x,y)
                        draw = True
                    else :
                        movement = False
                        break
                print("snake_head", self.snake_head_ids[0])
                print(f"Snake_part-{snake_id} : {x, y}, direction : {direction}, self.direction_is_diff : {self.direction_is_diff}")
                #dessin du snake
                if draw : 
                    canvas.coords(
                        snake_id,
                        self.grid_pos[x][y][0][1],
                        self.grid_pos[x][y][0][0],
                        self.grid_pos[x][y][1][1],
                        self.grid_pos[x][y][1][0],
                    )
            #update du canvas
            canvas.update()
            time.sleep(0.1)
        print(direction)

In [6]:
root = Tk()
snake_game = Snake_game(root)
root.mainloop()

Playing
apple .!canvas
snake .!canvas
coords [120.0, 100.0, 140.0, 120.0]
snake_head 2
Snake_part-3 : (8, 5), direction : right, self.direction_is_diff : False
coords [100.0, 100.0, 120.0, 120.0]
snake_head 2
Snake_part-4 : (7, 5), direction : right, self.direction_is_diff : False
coords [140.0, 100.0, 160.0, 120.0]
snake_head 2
Snake_part-2 : (6, 5), direction : right, self.direction_is_diff : False
coords [160.0, 100.0, 180.0, 120.0]
snake_head 2
Snake_part-3 : (9, 5), direction : right, self.direction_is_diff : False
coords [140.0, 100.0, 160.0, 120.0]
snake_head 2
Snake_part-4 : (8, 5), direction : right, self.direction_is_diff : False
coords [120.0, 100.0, 140.0, 120.0]
snake_head 2
Snake_part-2 : (7, 5), direction : right, self.direction_is_diff : False
coords [180.0, 100.0, 200.0, 120.0]
snake_head 2
Snake_part-3 : (10, 5), direction : right, self.direction_is_diff : False
coords [160.0, 100.0, 180.0, 120.0]
snake_head 2
Snake_part-4 : (9, 5), direction : right, self.direction_i

coords [100.0, 240.0, 120.0, 260.0]
snake_head 2
Snake_part-3 : (5, 11), direction : top, self.direction_is_diff : True
coords [100.0, 220.0, 120.0, 240.0]
snake_head 2
Snake_part-4 : (5, 10), direction : top, self.direction_is_diff : True
coords [100.0, 200.0, 120.0, 220.0]
snake_head 2
Snake_part-2 : (5, 9), direction : top, self.direction_is_diff : True
coords [100.0, 220.0, 120.0, 240.0]
snake_head 2
Snake_part-3 : (5, 10), direction : top, self.direction_is_diff : True
coords [100.0, 200.0, 120.0, 220.0]
snake_head 2
Snake_part-4 : (5, 9), direction : top, self.direction_is_diff : True
coords [100.0, 180.0, 120.0, 200.0]
snake_head 2
Snake_part-2 : (5, 8), direction : top, self.direction_is_diff : True
coords [100.0, 200.0, 120.0, 220.0]
snake_head 2
Snake_part-3 : (5, 9), direction : top, self.direction_is_diff : True
coords [100.0, 180.0, 120.0, 200.0]
snake_head 2
Snake_part-4 : (5, 8), direction : top, self.direction_is_diff : True
coords [100.0, 160.0, 120.0, 180.0]
snake_hea

coords [200.0, 360.0, 220.0, 380.0]
x>0
self.direction_is_diff False
snake_head 2
Snake_part-3 : (9, 18), direction : left, self.direction_is_diff : False
coords [180.0, 360.0, 200.0, 380.0]
x>0
self.direction_is_diff False
snake_head 2
Snake_part-4 : (8, 18), direction : left, self.direction_is_diff : False
coords [160.0, 360.0, 180.0, 380.0]
x>0
self.direction_is_diff False
snake_head 2
Snake_part-2 : (7, 18), direction : left, self.direction_is_diff : False
coords [180.0, 360.0, 200.0, 380.0]
x>0
self.direction_is_diff False
snake_head 2
Snake_part-3 : (8, 18), direction : left, self.direction_is_diff : False
coords [160.0, 360.0, 180.0, 380.0]
x>0
self.direction_is_diff False
snake_head 2
Snake_part-4 : (7, 18), direction : left, self.direction_is_diff : False
coords [140.0, 360.0, 160.0, 380.0]
x>0
self.direction_is_diff False
snake_head 2
Snake_part-2 : (6, 18), direction : left, self.direction_is_diff : False
coords [160.0, 360.0, 180.0, 380.0]
x>0
self.direction_is_diff False
s

coords [260.0, 240.0, 280.0, 260.0]
snake_head 2
Snake_part-3 : (13, 13), direction : bottom, self.direction_is_diff : True
coords [260.0, 260.0, 280.0, 280.0]
snake_head 2
Snake_part-4 : (13, 14), direction : bottom, self.direction_is_diff : True
coords [260.0, 280.0, 280.0, 300.0]
snake_head 2
Snake_part-2 : (13, 15), direction : bottom, self.direction_is_diff : True
coords [260.0, 260.0, 280.0, 280.0]
snake_head 2
Snake_part-3 : (13, 14), direction : bottom, self.direction_is_diff : True
coords [260.0, 280.0, 280.0, 300.0]
snake_head 2
Snake_part-4 : (13, 15), direction : bottom, self.direction_is_diff : True
coords [260.0, 300.0, 280.0, 320.0]
snake_head 2
Snake_part-2 : (13, 16), direction : bottom, self.direction_is_diff : True
coords [260.0, 280.0, 280.0, 300.0]
snake_head 2
Snake_part-3 : (13, 15), direction : bottom, self.direction_is_diff : True
coords [260.0, 300.0, 280.0, 320.0]
snake_head 2
Snake_part-4 : (13, 16), direction : bottom, self.direction_is_diff : True
coords [

coords [120.0, 200.0, 140.0, 220.0]
snake_head 2
Snake_part-3 : (7, 10), direction : right, self.direction_is_diff : False
coords [140.0, 200.0, 160.0, 220.0]
snake_head 2
Snake_part-4 : (8, 10), direction : right, self.direction_is_diff : False
coords [160.0, 200.0, 180.0, 220.0]
snake_head 2
Snake_part-2 : (9, 10), direction : right, self.direction_is_diff : False
coords [140.0, 200.0, 160.0, 220.0]
snake_head 2
Snake_part-3 : (8, 10), direction : right, self.direction_is_diff : False
coords [160.0, 200.0, 180.0, 220.0]
snake_head 2
Snake_part-4 : (9, 10), direction : right, self.direction_is_diff : False
coords [180.0, 200.0, 200.0, 220.0]
snake_head 2
Snake_part-2 : (10, 10), direction : right, self.direction_is_diff : False
coords [160.0, 200.0, 180.0, 220.0]
snake_head 2
Snake_part-3 : (9, 10), direction : right, self.direction_is_diff : False
coords [180.0, 200.0, 200.0, 220.0]
snake_head 2
Snake_part-4 : (10, 10), direction : right, self.direction_is_diff : False
coords [200.0,

...

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-3-ca8625e42ae2>", line 89, in <lambda>
    self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas))
  File "<ipython-input-3-ca8625e42ae2>", line 68, in set_key_event
    self.snake.move(self.direction, canvas, self.commands_history)
  File "<ipython-input-5-8afdcb87610e>", line 47, in move
    coords = canvas.coords(snake_id)
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 2469, in coords
    self.tk.call((self._w, 'coords') + args))]
_tkinter.TclError: invalid command name ".!canvas"
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 1705, in __call

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-3-ca8625e42ae2>", line 89, in <lambda>
    self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas))
  File "<ipython-input-3-ca8625e42ae2>", line 68, in set_key_event
    self.snake.move(self.direction, canvas, self.commands_history)
  File "<ipython-input-5-8afdcb87610e>", line 47, in move
    coords = canvas.coords(snake_id)
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 2469, in coords
    self.tk.call((self._w, 'coords') + args))]
_tkinter.TclError: invalid command name ".!canvas"
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 1705, in __call

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-3-ca8625e42ae2>", line 89, in <lambda>
    self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas))
  File "<ipython-input-3-ca8625e42ae2>", line 68, in set_key_event
    self.snake.move(self.direction, canvas, self.commands_history)
  File "<ipython-input-5-8afdcb87610e>", line 47, in move
    coords = canvas.coords(snake_id)
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 2469, in coords
    self.tk.call((self._w, 'coords') + args))]
_tkinter.TclError: invalid command name ".!canvas"
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 1705, in __call

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-3-ca8625e42ae2>", line 89, in <lambda>
    self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas))
  File "<ipython-input-3-ca8625e42ae2>", line 68, in set_key_event
    self.snake.move(self.direction, canvas, self.commands_history)
  File "<ipython-input-5-8afdcb87610e>", line 47, in move
    coords = canvas.coords(snake_id)
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 2469, in coords
    self.tk.call((self._w, 'coords') + args))]
_tkinter.TclError: invalid command name ".!canvas"
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 1705, in __call

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-3-ca8625e42ae2>", line 89, in <lambda>
    self.tinker.bind_all('<Key>', lambda event : self.set_key_event(event, self.canvas))
  File "<ipython-input-3-ca8625e42ae2>", line 68, in set_key_event
    self.snake.move(self.direction, canvas, self.commands_history)
  File "<ipython-input-5-8afdcb87610e>", line 47, in move
    coords = canvas.coords(snake_id)
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 2469, in coords
    self.tk.call((self._w, 'coords') + args))]
_tkinter.TclError: invalid command name ".!canvas"
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python37_64\lib\tkinter\__init__.py", line 1705, in __call